The following notebook takes all files in the /data folder and added them to a virtual raster named dtm.vrt it then extracts the boundery of this vrt and form the correction shape file it extracts all correction data that are within this boundary

In [18]:
from osgeo import gdal, ogr, osr
import subprocess
import os
from pathlib import Path
gdal.UseExceptions()
import geopandas as gpd
from shapely.geometry import box



In [5]:
dtm_root = Path('D:/GeoILUN/data/Lemvig')
corection_root = Path('D:/DK/tilpasninger')
src_root = Path('C:/Users/holmes/dev/semanticGIS/projects/GeoILUM/Processes/Hydro_ajust')


In [20]:
v_rast = dtm_root / 'dtm_10.tif'
horseshoe_file = corection_root / 'DHMHestesko.gpkg'
hs_filter = dtm_root / 'hs_filter.shp'
hs_with_z = dtm_root / 'hs_z.gpkg'
line_file = corection_root / 'DHMLinje.gpkg'
line_filter = dtm_root / 'line_filter.shp'
lines_with_z = dtm_root / 'line_z.gpkg'
burn_data = dtm_root / 'burn_z.gpkg'
hydro_dtm = dtm_root / 'hydro_dtm.tif'


## Buildes the Virtual raster

In [ ]:
!gdalbuildvrt {data_root}/dtm.vrt {data_root}/*.tif

## Extract the boundery

In [8]:
gdalSrc = gdal.Open(v_rast)
upx, xres, xskew, upy, yskew, yres = gdalSrc.GetGeoTransform()
cols = gdalSrc.RasterXSize
rows = gdalSrc.RasterYSize

In [22]:
print("Bounderybox North Max: %d , East min : %d" % (upx,upy))
print("Resolution X: %f, Y: %f" %(xres, yres))
print("Size X: %d, Y: %d" %(rows, cols))
print("Skew X: %f, Y:%f" %(xskew,yskew))

#Calculate alle four corners
ulx = upx + 0*xres + 0*xskew
uly = upy + 0*yskew + 0*yres
 
llx = upx + 0*xres + rows*xskew
lly = upy + 0*yskew + rows*yres
 
lrx = upx + cols*xres + rows*xskew
lry = upy + cols*yskew + rows*yres
 
urx = upx + cols*xres + 0*xskew
ury = upy + cols*yskew + 0*yres

# Create WKT for the bounding box
bbox_wkt = f"POLYGON(({ulx} {uly},{urx} {ury}, {lrx} {lry}, {llx} {lly}, {ulx} {uly}))"
bbox_geom = box(ulx, lly, urx, uly)
print("Boundery box: ", bbox_wkt)

Bounderybox North Max: 444506 , East min : 6289494
Resolution X: 10.000000, Y: -10.000000
Size X: 4899, Y: 3499
Skew X: 0.000000, Y:0.000000
Boundery box:  POLYGON((444506.0 6289494.0,479496.0 6289494.0, 479496.0 6240504.0, 444506.0 6240504.0, 444506.0 6289494.0))


# Extract horseshoe shooe data

In [ ]:


layer_name = 'dhmhestesko'



try:
    # 1. Create a bounding box geometry from your coordinates

    # 2. Read the specific layer from the GeoPackage into a GeoDataFrame
    # Geopandas automatically detects the geometry column.
    # We can use the 'bbox' parameter for an initial coarse filtering at the read stage.
    # This is highly efficient as it uses the spatial index of the GeoPackage.
    print(f"Reading geometries from '{horseshoe_file.name}' intersecting the bounding box...")
    gdf = gpd.read_file(
        horseshoe_file,
        layer=layer_name,
        bbox=bbox_geom
    )

    # 3. Check if any features were found
    if gdf.empty:
        print("No features found within the specified bounding box. No output file will be created.")
    else:
        # 4. Save the filtered GeoDataFrame to a Shapefile
        # The CRS is automatically carried over from the source GeoDataFrame.
        print(f"Found {len(gdf)} intersecting features. Writing to '{hs_filter.name}'...")
        gdf.to_file(hs_filter, driver='ESRI Shapefile')
        print("Successfully created shapefile.")

except Exception as e:
    print(f"An error occurred: {e}")


Reading geometries from 'DHMHestesko.gpkg' intersecting the bounding box...
Found 572 intersecting features. Writing to 'hs_filter.shp'...


C:\Users\holmes\AppData\Local\Temp\ipykernel_24188\602400767.py:31: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(hs_filter, driver='ESRI Shapefile')


Successfully created shapefile.


c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'datafordeleropdateringstid' to 'datafordel'
  ogr_write(
c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'id_namespace' to 'id_namespa'
  ogr_write(
c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'geometristatus' to 'geometrist'
  ogr_write(
c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'registreringsspecifikation' to 'registreri'
  ogr_write(
c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'forretningshaendelse' to 'forretning'
  ogr_write(
c:\Users\holmes\micromamba_files\envs\semanticGIS\Lib\sit

# Extract line data

In [11]:
geom_column = 'geom'
layer_name = 'dhmlinje'

base_name = os.path.splitext(os.path.basename(line_file))[0]
# SQL query using ST_Contains
sql_query = f"SELECT * FROM '{base_name}' WHERE ST_Contains(ST_GeomFromText('{bbox_wkt}'), geom)"

# Construct the ogr2ogr command
cmd = [
    'ogr2ogr',
    '-f', 'ESRI Shapefile',
    line_filter,
    line_file,
    '-dialect', 'SQLITE',
    '-sql', sql_query
]

# Execute the command
subprocess.run(cmd)

CompletedProcess(args=['ogr2ogr', '-f', 'ESRI Shapefile', WindowsPath('D:/GeoILUN/data/Lemvig/line_filter.shp'), WindowsPath('D:/DK/tilpasninger/DHMLinje.gpkg'), '-dialect', 'SQLITE', '-sql', "SELECT * FROM 'DHMLinje' WHERE ST_Contains(ST_GeomFromText('POLYGON((444506.0 6289494.0,479496.0 6289494.0, 479496.0 6240504.0, 444506.0 6240504.0, 444506.0 6289494.0))'), geom)"], returncode=1)

## Use the original Python scripts to first generate the data to be burned and then burn the data into the DTM

In [12]:
# Path to the original scripts
script_path = src_root / 'cli' / 'sample_line_z.py'

# Construct the command
command = ['python', script_path, v_rast, line_filter, lines_with_z]

# Execute the command
subprocess.run(command)

CompletedProcess(args=['python', WindowsPath('C:/Users/holmes/dev/semanticGIS/projects/GeoILUM/Processes/Hydro_ajust/cli/sample_line_z.py'), WindowsPath('D:/GeoILUN/data/Lemvig/dtm_10.tif'), WindowsPath('D:/GeoILUN/data/Lemvig/line_filter.shp'), WindowsPath('D:/GeoILUN/data/Lemvig/line_z.gpkg')], returncode=1)

In [13]:
# Path to the original scripts
script_path = src_root / 'cli' / 'sample_horseshoe_z_lines.py'

# Construct the command
command = ['python', script_path, v_rast, hs_filter, hs_with_z]

# Execute the command
subprocess.run(command)

CompletedProcess(args=['python', WindowsPath('C:/Users/holmes/dev/semanticGIS/projects/GeoILUM/Processes/Hydro_ajust/cli/sample_horseshoe_z_lines.py'), WindowsPath('D:/GeoILUN/data/Lemvig/dtm_10.tif'), WindowsPath('D:/GeoILUN/data/Lemvig/hs_filter.shp'), WindowsPath('D:/GeoILUN/data/Lemvig/hs_z.gpkg')], returncode=1)

In [14]:


def merge_geopackages(output_gpkg, input_gpkgs):
    # Ensure the output GeoPackage does not already exist
    driver = ogr.GetDriverByName('GPKG')
    if os.path.exists(output_gpkg):
        driver.DeleteDataSource(output_gpkg)
    
    out_ds = driver.CreateDataSource(output_gpkg)
    
    # Loop through each input GeoPackage
    for gpkg in input_gpkgs:
        in_ds = ogr.Open(gpkg)
        if not in_ds:
            print(f"Failed to open {gpkg}")
            continue

        # Copy each layer from the input GeoPackage to the output GeoPackage
        for i in range(in_ds.GetLayerCount()):
            in_layer = in_ds.GetLayerByIndex(i)
            if out_ds.GetLayer(in_layer.GetName()):
                print(f"Layer {in_layer.GetName()} already exists in the output GeoPackage.")
                continue
            # Copy layer to output GeoPackage, creating a new layer
            out_layer = out_ds.CopyLayer(in_layer, in_layer.GetName())
            if out_layer is None:
                print(f"Failed to copy layer {in_layer.GetName()} from {gpkg} to {output_gpkg}")

    out_ds = None  # Close the output data source to flush changes



# Usage
input_files = [lines_with_z, hs_with_z]
output_file = burn_data
merge_geopackages(output_file, input_files)


RuntimeError: D:\GeoILUN\data\Lemvig\line_z.gpkg: No such file or directory

In [ ]:

# Path to the original scripts
script_path = src_root / 'cli' / 'burn_line_z.py'

# Construct the command
command = ['python', script_path, burn_data, v_rast, hydro_dtm]

# Execute the command
subprocess.run(command)